In [2]:
import pandas as pd
import xlsxwriter
import os
import datetime

# find the files needed to process
hotel_file = ""
website_file = ""
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    if "member" in f.lower() and f[:2] != "~$" and "-" in f:
        hotel_file = f
    if "booking" in f.lower() and f[:2] != "~$" and "-" in f:
        website_file = f

msg = "Files detected:\n" + website_file + "\n" + hotel_file + '\n' + "Press Enter key to begin the process."
a = raw_input(msg)


# define functions
def same_name(booking_name, hotel_name):
    if hotel_name[:5] == 'name\n':
        hotel_name = hotel_name[5:]

    if ',' in hotel_name:
        hotel_name = hotel_name.split(",")
        hotel_name = hotel_name[1][:] + " " + hotel_name[0]

    if hotel_name == booking_name:
        # print("h:", hotel_name, "b:", booking_name)
        return True
    else:
        return False


######################################################################################
match = 0
canceled = 0
not_found = 0

# load and prepare the booking df
xl = pd.ExcelFile(website_file)
website_df = xl.parse(header=0, keep_default_na=False)

# remove empty spaces " " in column names
website_col_names = list(website_df)
print(website_col_names)
for name in website_col_names:
    a = name.split(" ")
    if a[-1] == "":
        b = " ".join(a[:-1])
    else:
        b = " ".join(a)
    website_df = website_df.rename(columns={name: b})
website_col_names = list(website_df)
print(website_col_names)

# pick columns that have ok
web_df = website_df[website_df['Status'] == 'ok']
print('booking df created')

# determine the column names
guest_col_name = ""
if "Guest name(s)" in list(web_df):
    guest_col_name = 'Guest name(s)'
    print("lower case name")
else:
    guest_col_name = 'Guest Name(s)'
    print("upper case Name")

# if guest name is empty, take it from booked by
for index, row in web_df.iterrows():
    if row[guest_col_name] == '':
        name = row['Booked by']
        # .encode('utf-8')
        guest_name = name.split(",")
        web_df.loc[index, guest_col_name] = guest_name[1][1:] + " " + guest_name[0]

##########################################
# loading the hotel df
xl = pd.ExcelFile(hotel_file)
hotel_df = xl.parse(header=0, keep_default_na=False, )
print(list(hotel_df))

# for index, row in hotel_df.iterrows():
#     print(index, row['Guest Name'], row['Arrival'], row['Nts'], row['Conf/Cxl#'],row['Unnamed: 10'])

#########################################
# book_col_name = list(web_df)[0]

print("begin comparison")

# create worksheet
workbook = xlsxwriter.Workbook('Expenses.xlsx')
worksheet = workbook.add_worksheet()
rowm = 0
coln = 0
worksheet.write(rowm, coln, "Confirmation Number")
worksheet.write(rowm, coln + 1, "CRS Number")
worksheet.write(rowm, coln + 2, "Name")
worksheet.write(rowm, coln + 3, "Booking.com Price")
worksheet.write(rowm, coln + 4, "Description")
rowm += 1

workbook_nf = xlsxwriter.Workbook('Customers Not found.xlsx')
worksheet_nf = workbook_nf.add_worksheet()
row_nf = 0

###################################################################
# check loop
good = 0
for index, row in web_df.iterrows():

    # initialzie what to compare
    name = row[guest_col_name]
    check_in_date = row['Check-in']
    check_out_date = row['Check-out']
    check_in_date = check_in_date.split("-")
    check_out_date = check_out_date.split("-")
    found = False

    # check if in ok, but different price
    for index_ok, row_ok in hotel_df.iterrows():
        # booking name format:good, hotel:not good

        if (same_name(name.lower(), row_ok['Guest Name'].lower())):
            found = True

            if row_ok['Unnamed: 10'] != "":
                print(name,"ROOM canceled", row_ok['Unnamed: 10'])

                worksheet.write(rowm, coln, row['Book number'])
                worksheet.write(rowm, coln + 1, row_ok['Conf/Cxl#'])
                worksheet.write(rowm, coln + 2, name)
                worksheet.write(rowm, coln + 3, row['Price'])
                worksheet.write(rowm, coln + 4, "Cancelled")

                print(name, " found in CANCEL file")
                rowm += 1
                canceled += 1
                break

            # if not found in cancel file, we check the date
            # hotel date
            date_format = "%y-%m-%d"
            arrival = str(row_ok['Arrival'])
            arrival = arrival.split(" ")

            h_arrival = datetime.datetime.strptime(arrival[0], date_format)
            h_departure = h_arrival + datetime.timedelta(days=row_ok['Nts'])


            # booking.com date
            b_checkin = datetime.datetime(int(check_in_date[0]), int(check_in_date[1]), int(check_in_date[2][:2]))
            b_checkout = datetime.datetime(int(check_out_date[0]), int(check_out_date[1]),
                                           int(check_out_date[2][:2]))
            found = True
            match += 1

            print('h_arrival: ', h_arrival)
            print('h_departure: ', h_departure)
            print("b_checkin: ", b_checkin)
            print("b_checkout", b_checkout)

            if not ((h_arrival == b_checkin) & (h_departure == b_checkout)):
                worksheet.write(rowm, coln, row['Book number'])
                worksheet.write(rowm, coln + 1, row_ok['CRSBookNum'])
                worksheet.write(rowm, coln + 2, name)
                worksheet.write(rowm, coln + 3, row['Price'])
                worksheet.write(rowm, coln + 4, "Checked in, but different date")
                print(name, " Found in OK file, but different date")
                rowm += 1
                break
            else:
                good += 1
                print(name," everything match!")
                break

    if found == False:
        # print(name, " Can't find customer Name")
        not_found += 1
        worksheet_nf.write(row_nf, 0, name)
        row_nf += 1
        print(name, " customer not found")

print('finished')
print("good:", good)
print("match: ", match)
print("canceled: ", canceled)
print("not found: ", not_found)
workbook.close()
workbook_nf.close()


Files detected:
booking arrival 9.1-30.xls
Member web arrival list 9.14-20.xlsx
Press Enter key to begin the process.
[u'Book number', u'Booked by', u'Guest name(s)', u'Check-in', u'Check-out', u'Booked on', u'Status', u'Rooms', u'Persons', u'Price', u'Commission %', u'Commission amount', u'Payment status', u'Payment method', u'Remarks']
[u'Book number', u'Booked by', u'Guest name(s)', u'Check-in', u'Check-out', u'Booked on', u'Status', u'Rooms', u'Persons', u'Price', u'Commission %', u'Commission amount', u'Payment status', u'Payment method', u'Remarks']
booking df created
lower case name
[u'Guest Name', u'BWR Member', u'Rms', u'Room-Type', u'Rate Cd', u'Book Date(MST)', u'Arrival', u'Nts', u'Conf/Cxl#', u'GTD', 'Unnamed: 10']
begin comparison
(u'Christian Cohen', ' customer not found')
(u'Jacquelyn Hopkins', ' customer not found')
(u'Hawazin Najjar', ' customer not found')
(u'shila sheedfar', ' customer not found')
(u'Andres Bohorquez', ' customer not found')
(u'Darin Tuggle', ' cust

ValueError: time data '2017-09-14' does not match format '%y-%m-%d'

In [3]:
###################################################################
# check loop
good = 0
for index, row in web_df.iterrows():

    # initialzie what to compare
    name = row[guest_col_name]
    check_in_date = row['Check-in']
    check_out_date = row['Check-out']
    check_in_date = check_in_date.split("-")
    check_out_date = check_out_date.split("-")
    found = False

    # check if in ok, but different price
    for index_ok, row_ok in hotel_df.iterrows():
        # booking name format:good, hotel:not good

        if (same_name(name.lower(), row_ok['Guest Name'].lower())):
            found = True

            if row_ok['Unnamed: 10'] != "":
                print(name,"ROOM canceled", row_ok['Unnamed: 10'])

                worksheet.write(rowm, coln, row['Book number'])
                worksheet.write(rowm, coln + 1, row_ok['Conf/Cxl#'])
                worksheet.write(rowm, coln + 2, name)
                worksheet.write(rowm, coln + 3, row['Price'])
                worksheet.write(rowm, coln + 4, "Cancelled")

                print(name, " found in CANCEL file")
                rowm += 1
                canceled += 1
                break

            # if not found in cancel file, we check the date
            # hotel date
            date_format = "%y-%m-%d"
            arrival = str(row_ok['Arrival'])
            arrival = arrival.split(" ")

            h_arrival = datetime.datetime.strptime(arrival[0], date_format)
            h_departure = h_arrival + datetime.timedelta(days=row_ok['Nts'])


            # booking.com date
            b_checkin = datetime.datetime(int(check_in_date[0]), int(check_in_date[1]), int(check_in_date[2][:2]))
            b_checkout = datetime.datetime(int(check_out_date[0]), int(check_out_date[1]),
                                           int(check_out_date[2][:2]))
            found = True
            match += 1

            print('h_arrival: ', h_arrival)
            print('h_departure: ', h_departure)
            print("b_checkin: ", b_checkin)
            print("b_checkout", b_checkout)

            if not ((h_arrival == b_checkin) & (h_departure == b_checkout)):
                worksheet.write(rowm, coln, row['Book number'])
                worksheet.write(rowm, coln + 1, row_ok['CRSBookNum'])
                worksheet.write(rowm, coln + 2, name)
                worksheet.write(rowm, coln + 3, row['Price'])
                worksheet.write(rowm, coln + 4, "Checked in, but different date")
                print(name, " Found in OK file, but different date")
                rowm += 1
                break
            else:
                good += 1
                print(name," everything match!")
                break

    if found == False:
        # print(name, " Can't find customer Name")
        not_found += 1
        worksheet_nf.write(row_nf, 0, name)
        row_nf += 1
        print(name, " customer not found")

print('finished')
print("good:", good)
print("match: ", match)
print("canceled: ", canceled)
print("not found: ", not_found)
workbook.close()
workbook_nf.close()


(u'Christian Cohen', ' customer not found')
(u'Jacquelyn Hopkins', ' customer not found')
(u'Hawazin Najjar', ' customer not found')
(u'shila sheedfar', ' customer not found')
(u'Andres Bohorquez', ' customer not found')
(u'Darin Tuggle', ' customer not found')
(u'Peter Vaccaro', ' customer not found')
(u'Desirae ONeill', ' customer not found')
(u'Haquika Howze', ' customer not found')
(u'Emily Shindler', ' customer not found')
(u'Nasir  Alqahtani', ' customer not found')
(u'Christine Dwyer', ' customer not found')
(u'Nemar Parchment', ' customer not found')
(u'Alex Wolf', ' customer not found')
(u'shanne acevedo', ' customer not found')
(u'Vincent Spinelli', ' customer not found')
(u'Haruhoko Nishie', ' customer not found')
(u'Mr.sultan Alosaimi', ' customer not found')
(u'Hakan Erkol', ' customer not found')
(u'Hernan Guillermo Berna', ' customer not found')
(u'Channel Mercado', ' customer not found')
(u'Kristina Darcangelo', ' customer not found')
(u'William Amos', ' customer not fo

ValueError: time data '2017-09-14' does not match format '%y-%m-%d'

In [4]:
arrival = str(row_ok['Arrival'])
            arrival = arrival.split(" ")
a=['2017-09-14', '00:00:00']

In [8]:
b=a[0].split("-")
b

['2017', '09', '14']

In [17]:
c="/".join([b[1],b[2],b[-0]])
c

'09/14/2017'